In [1]:
!pip install google-generativeai
!pip install python-dotenv

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import threading
import time
import concurrent

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
load_dotenv()
api_key_1 = os.getenv('GOOGLE_API_KEY_1')
api_key_2 = os.getenv('GOOGLE_API_KEY_2')
api_key_3 = os.getenv('GOOGLE_API_KEY_3')
api_key_4 = os.getenv('GOOGLE_API_KEY_4')
api_key_5 = os.getenv('GOOGLE_API_KEY_5')
api_key_6 = os.getenv('GOOGLE_API_KEY_6')

api_keys = [
    api_key_1,
    api_key_2,
    api_key_3,
    api_key_4,
    api_key_5,
    api_key_6
]

In [4]:
filenames = [
    "group_1.txt",
    "group_2.txt",
    "group_3.txt",
    "group_4.txt",
    "group_5.txt",
    "group_6.txt",
    "group_7.txt",
    "group_8.txt",
    "group_9.txt",
    "group_10.txt",
    "group_11.txt",
    "group_12.txt",
]

In [5]:
file_contents = []

In [6]:
for filename in filenames:
    file = open(f"./data/{filename}", "r")
    file_content = file.read()
    file.close()
    file_contents.append(file_content)

In [7]:
question = "Kas ma võin autoga sõites alkoholi tarbida?"

In [8]:
system_prompt = "Sa oled seaduste abiline. Kui sulle antud seaduses on küsimusele vastust, siis vasta sellele. Kui vastus puudub, siis vasta '0'"

In [9]:
response_format = """kastuta vastamisel järgmist formaati:
<seaduse nimi>

<sinu vastus>"""

In [10]:
responses = []

In [16]:
#group_index = 0
#group_count = len(api_keys) * 2

#for api_key in api_keys:
#    genai.configure(api_key=api_key)
#
#    print(f"progress: {group_index + 1}/{group_count}")
#    file_content = file_contents[group_index]
#    model = genai.GenerativeModel("gemini-1.5-flash")
#    response_text = model.generate_content([
#        system_prompt,
#        response_format,
#        file_content,
#        "Küsimus on järgmine:",
#        question
#    ]).text
#    responses.append(response_text)

#    group_index += 1

#    print(f"progress: {group_index + 1}/{group_count}")
#    file_content = file_contents[group_index]
#    model = genai.GenerativeModel("gemini-1.5-flash-8b")
#    response_text = model.generate_content([
#        system_prompt,
#        response_format,
#        file_content,
#        "Küsimus on järgmine:",
#        question
#    ]).text
#    responses.append(response_text)

#    group_index += 1

In [91]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    from: https://stackoverflow.com/questions/3173320/text-progress-bar-in-terminal-with-block-characters
    
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
thread_local = threading.local()
responses_lock = threading.Lock()
api_keys_lock = threading.Lock()
responses = []


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get_response(data):
    api_keys, file_content, group_index, group_count, system_prompt, response_format, question = data
    
    with api_keys_lock:
        if not api_keys:
            print("No API keys left to process.")
            return
        api_key = api_keys.pop(0)
        
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel("gemini-1.5-flash")
    if group_index % 2 == 0:
        model = genai.GenerativeModel("gemini-1.5-flash-8b")


    time.sleep(2)

    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text

    # Safely append to the shared responses list
    with responses_lock:
        responses.append((group_index, response_text))
        
    groups_done = len(responses)
    printProgressBar(groups_done, group_count, length = 50)

def get_all_responses(api_keys, file_contents, system_prompt, response_format, question):
    group_count = len(api_keys)
    # print(group_count)
    tasks = [
        ( api_keys, file_contents[i], i, group_count, system_prompt, response_format, question)
        for i in range(group_count)
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        executor.map(get_response, tasks)

if __name__ == "__main__":

    temp_api_keys = api_keys.copy()
    temp_api_keys = temp_api_keys * 2
    temp_api_keys.sort()

    start_time = time.time()

    group_count = len(api_keys)
    printProgressBar(0, group_count, length = 50)

    get_all_responses(temp_api_keys, file_contents, system_prompt, response_format, question)
    
    duration = time.time() - start_time

    responses.sort(key=lambda x: x[0])
    for idx, response_text in responses:
        print(f"Response {idx + 1}: {response_text}")

    print(f"I did it in {duration} seconds")


In [46]:
for response in responses:
    print(response)

(0, '<Karistusseadustik>\n\n0\n')
(1, 'Liiklusseadus\n\nEi, autoga sõites ei tohi alkoholi tarbida.  Käesoleva seaduse § 69 sätestab, et juht ei tohi olla alkoholi piirmäära ületavas seisundis ega joobeseisundis.\n')
(2, '<Karistusseadustik>\n\n0\n')
(3, 'Meresõiduohutuse seadus\n\nEi, Meresõiduohutuse seaduse § 45 lõike 4 ja § 37 lõike 2 alusel on väikelaeva või muu veesõiduki juhtimisel keelatud olla joobeseisundis.  Alkoholi tarvitamine autoga sõitmisel ei ole otseselt reguleeritud, aga kuna sõidukijuht peab olema alati võimeline sõidukit ohutult juhtima, siis ei ole alkoholi tarvitamine sõidu ajal soovitatav.\n')
(4, '<Karistusseadustiku §-d>\n\n0\n')
(6, '<Korrakaitseseadus>\n\n0\n')
(7, 'Liikluskindlustuse seadus\n\nEi, liikluskindlustuse seaduse § 8 lõike 2 punkti 4 alusel ei ole lubatud juhtida sõidukit joobeseisundis.\n')
(8, 'Alkoholiseadus\n\n0\n')
(9, 'Liiklusseadus\n\n0\n')
(10, '<Alkoholi-, tubaka-, kütuse- ja elektriaktsiisi seadus>\n\n0\n')
(11, '0\n')


In [47]:
genai.configure(api_key=api_keys[0])
model = genai.GenerativeModel("gemini-1.5-pro")
response_texts = [element[1] for element in responses]

prompt = (
    ["Sulle antakse mitme erineva mudeli vastused erinevatest seadustest, tsiteeri mulle mudeli vastuseid, mis pole '0'"] +
    [response_format] +
    response_texts + 
    ["Küsimus on järgnev:"] +
    [question]
)

best_answer = model.generate_content(prompt)

In [48]:
print(best_answer.text)

Liiklusseadus

Ei, autoga sõites ei tohi alkoholi tarbida.  Käesoleva seaduse § 69 sätestab, et juht ei tohi olla alkoholi piirmäära ületavas seisundis ega joobeseisundis.
Meresõiduohutuse seadus

Ei, Meresõiduohutuse seaduse § 45 lõike 4 ja § 37 lõike 2 alusel on väikelaeva või muu veesõiduki juhtimisel keelatud olla joobeseisundis.  Alkoholi tarvitamine autoga sõitmisel ei ole otseselt reguleeritud, aga kuna sõidukijuht peab olema alati võimeline sõidukit ohutult juhtima, siis ei ole alkoholi tarvitamine sõidu ajal soovitatav.
Liikluskindlustuse seadus

Ei, liikluskindlustuse seaduse § 8 lõike 2 punkti 4 alusel ei ole lubatud juhtida sõidukit joobeseisundis.



---
# Testing chat ui
---

In [ ]:
import os
import time

# tell the chatbot what its purpose is about
messages = [
    {"role": "system", "content" : """You are my weekly mental health doctor and will help me maintain a positive mindset. 
            You will fill me with realistic and optimistic advice to get me through the work week. 
            Remember that I work from 9am to 6pm and rest of the day is with my family."""}
]

def generate_response(user_message):
    time.sleep(2)

    messages.append( {
        "role" : "assistant",
        "content": "fgsdfgsdfgdfg"
    })
    return "fgsdfgsdfgdfg"

print("Chatbot: Hello! Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    print
    response = generate_response(user_input)
    print(f"\033[0;32mChatbot: {response}\033[0m")